In [1]:
import pandas as pd
import geopandas as gpd

Replace the following file with your dataset.

In [2]:
FILENAME = "data/lstGAF.csv"

In [ ]:
df = pd.read_csv(FILENAME)
df.head(5) # can't plot it because it contains the GAF scores (privacy issues)

In [4]:
final_cols = ["pt", "X", "Y", "GAF", "GAF_ADJ"]

Create list of sub data frames for each year and mean and median. 

In [5]:
years = [1998, 2008, 2018]
select_columns = []
for year in years:
    for m in ["mean", "median"]:
        tmp = []
        for col in df.columns:
            if str(year) in col and m in col.lower():
                tmp.append(col)
        select_columns.append(tmp)

In [6]:
tmp_frames = []
for col in select_columns:
    sel_cols = final_cols + col
    tmp_frames.append(df[sel_cols].copy())

In [7]:
for frame in tmp_frames:
    frame.dropna(inplace=True)

In [8]:
dfs = [tmp.set_index('pt') for tmp in tmp_frames]

Clean duplicated rows and cols

In [9]:
final = pd.concat(dfs, axis=1)
final.drop_duplicates(inplace=True)
final = final.loc[:,~final.columns.duplicated()].copy()

Convert K to °C

In [10]:
lst_cols = [col for col in final.columns if "lst" in col.lower()]
final[lst_cols] -= 273.15

Calculate Delta

In [11]:
final["DelaLstMedian9818"] = final.lstMedian2018 - final.lstMedian1998
final["DelaLstMean9818"] = final.lstMean2018 - final.lstMean1998
final["DelaLstMedian0818"] = final.lstMedian2018 - final.lstMedian2008
final["DelaLstMean0818"] = final.lstMean2018 - final.lstMean2008

# NDVI
final["DelaNDVIMedian9818"] = final.NDVIMedian2018 - final.NDVIMedian1998
final["DelaNDVIMean9818"] = final.NDVIMean2018 - final.NDVIMean1998
final["DelaNDVIMedian0818"] = final.NDVIMedian2018 - final.NDVIMedian2008
final["DelaNDVIMean0818"] = final.NDVIMean2018 - final.NDVIMean2008

Save to CSV

In [12]:
final.to_csv("data/finalCleanMerged.csv")

For geopackage, use the following code:

In [13]:
geoframe = gpd.GeoDataFrame(final, geometry=gpd.points_from_xy(final.X, final.Y))

In [14]:
geoframe.crs = "EPSG:4326"
geoframe.to_crs(epsg=21781, inplace=True)

In [15]:
geoframe.to_file("data/finalClean.gpkg", driver="GPKG")